In [81]:
import tensorflow as tf
from tensorflow import keras
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.4.1


In [56]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, f1_score, cohen_kappa_score
import matplotlib as mpl
import matplotlib.pyplot as plt
# import seaborn as sns

In [3]:
from os.path import join
import os

In [4]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [5]:
data_folder = "../../data/preprocessed/"

In [82]:
X = pd.read_pickle(join(data_folder, "X.pk"))
y = pd.read_pickle(join(data_folder, "Y.pk"))
X_mat = X.values
y_vec = y.values.flatten()
scaler = StandardScaler()
scaler.fit(X_mat)
X_mat = scaler.transform(X_mat)

x_train, x_test, y_train, y_test = train_test_split(X_mat, y_vec, test_size=0.12, random_state=123)


In [83]:
np.sum(y_test == 2)

7899

In [84]:
x_test.shape

(8486, 127)

In [85]:
EPOCHS = 100
BATCH_SIZE = 2048


In [99]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.15),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(3)
])

In [100]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.02772392, -0.04086035,  0.09623041]], dtype=float32)

In [101]:
tf.nn.softmax(predictions).numpy()

array([[0.33282053, 0.31075945, 0.35642004]], dtype=float32)

In [102]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# sample_weight=tf.constant([2, 2, 0.1])
loss_fn(y_train[:1], predictions, sample_weight=tf.constant([2, 2, 0.1])).numpy()

1.5972728

In [105]:
model.compile(optimizer='adam',
              # loss=loss_fn,
              loss=loss_fn,
              metrics=['categorical_accuracy'])

In [106]:
class_weight = {0:1.2,
               1:1.2,
               2:0.1}
model.fit(x_train, y_train, epochs=175, class_weight=class_weight)

Epoch 1/175
1945/1945 [==============================] - 3s 1ms/step - loss: 0.1286 - categorical_accuracy: 0.0343
Epoch 2/175
1945/1945 [==============================] - 3s 1ms/step - loss: 0.1209 - categorical_accuracy: 0.0354
Epoch 3/175
1945/1945 [==============================] - 3s 1ms/step - loss: 0.1210 - categorical_accuracy: 0.0340
Epoch 4/175
1945/1945 [==============================] - 3s 1ms/step - loss: 0.1229 - categorical_accuracy: 0.0332
Epoch 5/175
1945/1945 [==============================] - 2s 1ms/step - loss: 0.1212 - categorical_accuracy: 0.0341
Epoch 6/175
1945/1945 [==============================] - 3s 1ms/step - loss: 0.1169 - categorical_accuracy: 0.0348
Epoch 7/175
1945/1945 [==============================] - 3s 1ms/step - loss: 0.1227 - categorical_accuracy: 0.0334
Epoch 8/175
1945/1945 [==============================] - 3s 1ms/step - loss: 0.1159 - categorical_accuracy: 0.0334
Epoch 9/175
1945/1945 [==============================] - 3s 1ms/step - loss: 0.1

In [107]:
model.evaluate(x_test,  y_test, verbose=2)



266/266 - 0s - loss: 0.5350 - categorical_accuracy: 0.0152


[0.5350283980369568, 0.015201508067548275]

In [109]:
logits = model.predict(x_test)
results = tf.math.argmax(tf.nn.softmax(logits).numpy(), axis =1)

In [110]:
cm = confusion_matrix(results, y_test)
score2 = cohen_kappa_score(results, y_test)

In [111]:
cm

array([[  14,   20,   95],
       [  36,   40,  605],
       [ 189,  288, 7199]])

In [98]:
score2

0.033693250556926735

## New Predictions

In [71]:
X = pd.read_pickle(join(data_folder, "test.pk"))
X_mat = X.values

In [72]:
X_mat = scaler.transform(X_mat)

In [74]:
logits = model.predict(X_mat)
results = tf.math.argmax(tf.nn.softmax(logits).numpy(), axis =1)

In [79]:
predicted_vals = results.numpy()

In [80]:

report = pd.DataFrame({'Id': X.index, 'pred': predicted_vals})
report.to_csv("submission.csv", index=False)